# <c> Обработка выгрузок Норм сбережения населения </c>

In [264]:
import pandas as pd
import numpy as np
import warnings
from pandas. tseries.offsets import DateOffset
from dateutil.relativedelta import relativedelta
warnings.filterwarnings('ignore')

In [265]:
# делаем столбец с индексами регионов если не находятся то ищем совпадения по ФО
def make_region_index(row):
    for ind in range(len(regions_dict)):
        if regions_dict.loc[ind,'region_name'] in row:
            return regions_dict.loc[ind,'region_id']#, regions_dict.loc[ind,'state_id']
    for ind in range(len(states_dict)):
        if states_dict.loc[ind,'state_name'] in row:
            return states_dict.loc[ind,'state_id']
        
# из ЕМИСС мы получаем квартальные сведегния с названиями столбцов в 2 строки
# функция выискивает римсие названия кварталов и меняет их на одномерные прибавляя
# к году (первый элемент многострочного столбца) месяц квартала
def make_year_quarter_col(df):
    date_range = []
    for i in range(0, len(df.columns)):
        if 'IV' in df.columns[i][1]:
            date_range.append(str(df.columns[i][0])+'-12-01')
        elif 'III' in df.columns[i][1]:
            date_range.append(str(df.columns[i][0])+'-09-01')
        elif 'II' in df.columns[i][1]:
            date_range.append(str(df.columns[i][0])+'-06-01')  
        elif 'I' in df.columns[i][1]:
            date_range.append(str(df.columns[i][0])+'-03-01')
    print('First:', date_range[0], 'Last:', date_range[-1])
    df.columns = date_range
    return df
        
# Подгоняем формат и количество столбцов выгрузок ЕМИСС под общий вид
# Убираем столбцы с названием показателя и единицами измерения
# Удаляем мусорную пеорвую строку и переименовываем первый столбец в region
def format_yearly_df(df):
    df.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2'], axis=1, inplace=True)
    df = df.iloc[1:,:]
    #df = df.rename(columns={'Unnamed: 1':'region'})
    df.columns = df.columns.astype('datetime64[ns]')
    df.reset_index(drop=True, inplace=True)
    return df

def format_quartary_df(df):
    df.drop(['Unnamed: 0_level_0', 
             'Unnamed: 1_level_0', 
             'Unnamed: 2_level_0'],
            axis=1, inplace=True)
    #df = df.rename(columns={'Unnamed: 1':'region'})
    date_range = []
    for i in range(0, len(df.columns)):
        if 'IV' in df.columns[i][1]:
            date_range.append(str(df.columns[i][0])+'-12-01')
        elif 'III' in df.columns[i][1]:
            date_range.append(str(df.columns[i][0])+'-09-01')
        elif 'II' in df.columns[i][1]:
            date_range.append(str(df.columns[i][0])+'-06-01')  
        elif 'I' in df.columns[i][1]:
            date_range.append(str(df.columns[i][0])+'-03-01')
    print('First:', date_range[0], 'Last:', date_range[-1])
    df.columns = date_range
    df.columns = df.columns.astype('datetime64[ns]')
    df.reset_index(drop=True, inplace=True)
    # df.insert(column='region_id', 
    #         loc=0, 
    #         value=[x for x in range(96)])
    return df

In [266]:
# Загрузка словарей кодов субъектов и округов
regions_dict = pd.read_excel("D:/ECONOMICS/regions.xlsx", 
                             sheet_name='regions_short', 
                             header=0)
states_dict = pd.read_excel("D:/ECONOMICS/regions.xlsx", 
                             sheet_name='states_short', 
                             header=0)
regions_seq = pd.read_excel("D:/ECONOMICS/regions.xlsx", 
                             sheet_name='regions_seq', 
                             header=0)

In [267]:
ind_y = {}
ind_q = {}

In [268]:
# Прирост уменьшение денег на руках населений. Наличные год ind070
ind_y['IND0070'] = pd.read_excel("D:/ECONOMICS/savings/src1/Наличные_год.xls", header=2)

In [269]:
ind_y['IND0070'] = format_yearly_df(ind_y['IND0070'])

In [270]:
#ind_y['IND0070'].drop(['Unnamed: 0','Unnamed: 1','Unnamed: 2'], axis=1, inplace=True)
#ind_y['IND0070'] = ind_y['IND0070'].iloc[1:,:]
# не используем функцию переименования по номерам регионов по словарю т.к. 
# решили сделать сквозную нумерацию из-за того что в ЕМИСС и данных банковского сектора
# (БС) ЦБ последовательность регионов совпадает.
# по той же причине переименовывать столбец для поиска по словарю нет необходимости
#ind_cash_y = ind_cash_y.rename(columns={'Unnamed: 1':'region'})
#ind_cash_y['region_id'] = ind_cash_y.region.apply(make_region_index)
# ind_y['IND0070'].insert(column='region_id', 
#                   loc=0, 
#                   value=[x for x in range(96)])

def insert_region_idseq(df):
    df.insert(column='region_id', 
                  loc=0, 
                  value=[x for x in range(96)])
    return df

In [271]:
# Прирост уменьшение денег на руках населений. Наличные Q ind010
ind_q['IND0010'] = pd.read_excel("D:/ECONOMICS/savings/src1/Наличные_кв.xls", header=[2,3])
ind_q['IND0010'] = format_quartary_df(ind_q['IND0010'])

First: 2019-03-01 Last: 2022-12-01


In [272]:
# удаляем все столбцы не несущие значений, по маске регион так же не нужен
# ind_q['IND0010'].drop(['Unnamed: 0_level_0', 
#                        'Unnamed: 1_level_0', 
#                        'Unnamed: 2_level_0'], 
#                         axis=1, inplace=True)
#ind_cash_q = ind_cash_q.rename(columns={'Unnamed: 1':'region'})

In [273]:
# ОБЯЗАТЕЛЬНЫЕ ПЛАТЕЖИ
# ind030 Q, ind090 Y
ind_y['IND0090'] = pd.read_excel("D:/ECONOMICS/savings/src1/Обязательные платежи_год.xls", header=2)
ind_q['IND0030'] = pd.read_excel("D:/ECONOMICS/savings/src1/Обязательные платежи_кв.xls", header=[2,3])
# ТОВАРЫ И УСЛУГИ
# ind020 Q, ind080 Y
ind_y['IND0080'] = pd.read_excel("D:/ECONOMICS/savings/src1/Товары и услуги_год.xls", header=2)
ind_q['IND0020'] = pd.read_excel("D:/ECONOMICS/savings/src1/Товары и услуги_кв.xls", header=[2,3])

In [274]:
#вызываем функцию приведения годовых показателей ЕМИСС к стандартному виду
ind_y['IND0090'] = format_yearly_df(ind_y['IND0090'])
#ind_payments_y = insert_region_idseq(ind_payments_y)

ind_y['IND0080'] = format_yearly_df(ind_y['IND0080'])
#ind_goods_serv_y = insert_region_idseq(ind_goods_serv_y)

ind_q['IND0030'] = format_quartary_df(ind_q['IND0030'])
ind_q['IND0020'] = format_quartary_df(ind_q['IND0020'])
# # вставляем столбец id региона
# ind_payments_y.insert(loc=1, column='region_id',
#                       value=ind_payments_y.region.apply(make_region_index))

First: 2019-03-01 Last: 2022-12-01
First: 2019-03-01 Last: 2022-12-01


In [275]:
# РДД Q ind130
# Среднедушевые ДД Q ind140
ind_q['IND0130'] = pd.read_excel("D:/ECONOMICS/savings/src1/РДД.xls", header=[2,3])
ind_q['IND0140'] = pd.read_excel("D:/ECONOMICS/savings/src1/Среднедушевые ДД.xls", header=[2,3])

ind_q['IND0130'] = format_quartary_df(ind_q['IND0130'])
ind_q['IND0140'] = format_quartary_df(ind_q['IND0140'])

#Численность постоянного населения Y ind150
ind_y['IND0150'] = pd.read_excel("D:/ECONOMICS/savings/src1/Численность постоянного населения.xls", header=2)
ind_y['IND0150'] = format_yearly_df(ind_y['IND0150'])

First: 2019-03-01 Last: 2022-12-01
First: 2020-03-01 Last: 2022-12-01


In [277]:
# ПОДГРУЗКА РЕТРО
ind_retro_m = pd.read_excel("D:/ECONOMICS/savings/src1/out/СВОД_НС.xlsx", 
                            header=0, 
                            sheet_name='month')
ind_retro_q = pd.read_excel("D:/ECONOMICS/savings/src1/out/СВОД_НС.xlsx", 
                            header=0, 
                            sheet_name='quarter')
ind_retro_y = pd.read_excel("D:/ECONOMICS/savings/src1/out/СВОД_НС.xlsx", 
                            header=0, 
                            sheet_name='year')

In [278]:
def extract_retro(df):
    df_tuple ={}
    for ind in df.name_code.unique():
        df_tuple[ind] = df[df.name_code==ind]
        df_tuple[ind].drop(['name_code','region_code'], axis=1, inplace=True)
        df_tuple[ind].reset_index(drop=True, inplace=True)
        df_tuple[ind].dropna(axis=1, how='all', inplace=True)
        #print(df_tuple[ind].columns)
        df_tuple[ind].columns = pd.to_datetime(df_tuple[ind].columns, format='%Y') #.astype('datetime64[ns]')
        #print(pd.to_datetime(df_tuple[ind].columns, format='%Y'))
    return df_tuple

def extract_retro_m(df):
    df_tuple ={}
    for ind in df.name_code.unique():
        df_tuple[ind] = df[df.name_code==ind]
        df_tuple[ind].drop(['name_code','region_code'], axis=1, inplace=True)
        df_tuple[ind].reset_index(drop=True, inplace=True)
        df_tuple[ind].dropna(axis=1, how='all', inplace=True)
        #print(df_tuple[ind].columns)
        df_tuple[ind].columns = pd.to_datetime(df_tuple[ind].columns, format='%Y-%m-%d') #.astype('datetime64[ns]')
        #print(pd.to_datetime(df_tuple[ind].columns, format='%Y'))
    return df_tuple

In [279]:
# Разбиваем лист исходного периода на показатели в tuple 
# убираем название (индекс) самого показателя и перекидывем его в название 
ind_retro_q = extract_retro(ind_retro_q)
ind_retro_y = extract_retro(ind_retro_y)
ind_retro_m = extract_retro_m(ind_retro_m)

In [280]:
def concat_corr(df_retro, df_new):
    # Удаляем из исходника все столбцы с датами которые есть в новом файле
    # и дописываем новый файл в конец tuple
    for ind in df_new:
        new_dates = (pd.to_datetime(df_new[ind].columns, format='%y-%m-%d'))
        #display(new_dates)
        #display(df_retro[ind])
        df_retro[ind] = df_retro[ind].loc[:,~df_retro[ind].columns.isin(new_dates)]
        ###df_retro[ind].drop(columns=remove_dates)
        df_retro[ind] = pd.concat([df_retro[ind], df_new[ind]], axis=1)
        #df_retro[ind].insert(column='region_code', loc=0, value=df_retro[ind].index)
        #df_retro[ind].insert(column='IND', loc=0, value=ind)
        #display(df_retro[ind])
    return df_retro

In [281]:
ind_retro_q = concat_corr(ind_retro_q, ind_q)
ind_retro_y = concat_corr(ind_retro_y, ind_y)

In [283]:
ind_retro_q['IND0040'] = 100 - (ind_retro_q['IND0010']+\
                                ind_retro_q['IND0020']+\
                                ind_retro_q['IND0030'])
ind_retro_q['IND0050'] = ind_retro_q['IND0010']+ind_retro_q['IND0040']

ind_retro_q['IND0060'] = ind_retro_q['IND0050']/(100 - ind_retro_q['IND0030'])*100

In [284]:
ind_retro_y['IND0100'] = 100 - (ind_retro_y['IND0070']+ind_retro_y['IND0080']+ind_retro_y['IND0090'])
ind_retro_y['IND0110'] = ind_retro_y['IND0070']+ind_retro_y['IND0100']
ind_retro_y['IND0120'] = ind_retro_y['IND0110'] / (100 - ind_retro_y['IND0090'])*100

In [285]:
ind_retro_q['IND0160'] = pd.DataFrame()#index = ind_retro_q['IND0140'].index , columns=ind_retro_q['IND0140'].columns)
for yr in ind_retro_y['IND0150'].columns.year:
    #ind_retro_q['IND0160'] = ind_retro_q['IND0160'].iloc[0,0]
    #ind_retro_q[ind_retro_q['IND0140'].columns.year == yr] * ind_retro_y[ind_retro_y['IND0150'].columns.year==yr]
    #ind_retro_y[ind_retro_y['IND0150'].columns.year==yr]
    #print(ind_retro_y['IND0150'].loc[:,ind_retro_y['IND0150'].columns.year==yr])
    #display(ind_retro_q['IND0140'].loc[:,ind_retro_q['IND0140'].columns.year == yr])
    curr_year = ind_retro_q['IND0140'].\
                    loc[:,ind_retro_q['IND0140'].columns.year == yr] *\
                ind_retro_y['IND0150'].\
                    loc[:,ind_retro_y['IND0150'].columns.year==yr].to_numpy()\
                / 1000000
    #ind_retro_q['IND0160'].loc[:,ind_retro_q['IND0160'].columns.year == yr] = curr_year
    #display(ind_retro_q['IND0160'].loc[:,ind_retro_q['IND0160'].columns.year == yr])
    ind_retro_q['IND0160'] = pd.concat([ind_retro_q['IND0160'], curr_year], axis=1)
    #display(ind_retro_q['IND0140'].loc[:,ind_retro_q['IND0140'].columns.year == yr] * ind_retro_y['IND0150'].loc[:,ind_retro_y['IND0150'].columns.year==yr])
    #display(ind_retro_q['IND0140'].loc[:,ind_retro_q['IND0140'].columns.year == yr])
    #display(ind_retro_y['IND0150'].loc[:,ind_retro_y['IND0150'].columns.year==yr].to_numpy())

In [286]:
#ind_retro_m['IND0210'].columns = ind_retro_m['IND0210'].columns-pd.DateOffset(months=1)

In [287]:
# расчет накопленного итога 01.02 - это январь соответствующего года. делим месяц на 2 чтобы 
# в февраль (он же янв)
ind_retro_m['IND0220'] = ind_retro_m['IND0210'].copy()
for mnth_n in range(2, len(ind_retro_m['IND0210'].columns)):
    if ind_retro_m['IND0210'].columns[mnth_n].month/2.0!=1.0:
        ind_retro_m['IND0220'].iloc[:,mnth_n] = ind_retro_m['IND0220'].iloc[:,mnth_n] + ind_retro_m['IND0220'].iloc[:,mnth_n-1]

In [289]:
def calc_deposits_inc(ind_base_df):
    q1 = ind_base_df.loc[:,ind_base_df.columns.month == 4].T.reset_index(drop=True) -ind_base_df.loc[:,ind_base_df.columns.month == 1].T.reset_index(drop=True)
    q1 = q1.T
    q1.columns= (ind_base_df.loc[:,ind_base_df.columns.month == 1].columns+pd.DateOffset(month=3))
    
    q2 = ind_base_df.loc[:,ind_base_df.columns.month == 7].T.reset_index(drop=True) -ind_base_df.loc[:,ind_base_df.columns.month == 4].T.reset_index(drop=True)
    q2 = q2.T
    q2.columns= (ind_base_df.loc[:,ind_base_df.columns.month == 4].columns+pd.DateOffset(month=6))
    
    q3 = ind_base_df.loc[:,ind_base_df.columns.month == 10].T.reset_index(drop=True) -ind_base_df.loc[:,ind_base_df.columns.month == 7].T.reset_index(drop=True)
    q3 = q3.T
    q3.columns= (ind_base_df.loc[:,ind_base_df.columns.month == 7].columns+pd.DateOffset(month=9))
    
    q4 = ind_base_df.loc[:,ind_base_df.columns.month == 1].iloc[:,1:].T.reset_index(drop=True) -ind_base_df.loc[:,ind_base_df.columns.month == 10].T.reset_index(drop=True)
    q4 = q4.T
    q4.columns= (ind_base_df.loc[:,ind_base_df.columns.month == 10].columns+pd.DateOffset(month=12))
    
    quartary_all = pd.concat([q1, q2, q3, q4], axis=1)
    quartary_all = quartary_all[sorted(quartary_all.columns)]

    return quartary_all

In [290]:
ind_retro_q['IND0230'] = calc_deposits_inc(ind_retro_m['IND0170'])
ind_retro_q['IND0240'] = calc_deposits_inc(ind_retro_m['IND0180'])
ind_retro_q['IND0250'] = calc_deposits_inc(ind_retro_m['IND0190'])

In [326]:
def qoq_from_monthly(src_ind):
    quartary = src_ind.loc[:,src_ind.columns.month.isin([1, 4, 7, 10])]
    #выбрали с месяца только столбцы с месяцем 1, 4, 7, 10 и смещаем их на -1 т.к. это показатели квартала
    quartary.columns = quartary.columns - pd.DateOffset(months=1)
    # считаем QoQ
    qoq = quartary.copy()
    for ind in range(5, len(quartary.columns)):
        qoq.iloc[:,ind] = (quartary.iloc[:,ind] / quartary.iloc[:,ind-4]) *100
    return qoq

In [332]:
ind_retro_q['IND0260'] = qoq_from_monthly(ind_retro_m['IND0170'])
ind_retro_q['IND0270'] = qoq_from_monthly(ind_retro_m['IND0220'])
ind_retro_q['IND0280'] = qoq_from_monthly(ind_retro_m['IND0200'])

In [339]:
def trim_to_2018(df):
    df = df.loc[:, df.columns>'2017-12-01']
    return df

In [340]:
ind_retro_q['IND0230'] = trim_to_2018(ind_retro_q['IND0230'])
ind_retro_q['IND0240'] = trim_to_2018(ind_retro_q['IND0240'])
ind_retro_q['IND0250'] = trim_to_2018(ind_retro_q['IND0250'])
ind_retro_q['IND0260'] = trim_to_2018(ind_retro_q['IND0260'])
ind_retro_q['IND0270'] = trim_to_2018(ind_retro_q['IND0270'])
ind_retro_q['IND0280'] = trim_to_2018(ind_retro_q['IND0280'])

In [370]:
ind_retro_q['IND0290'] = (ind_retro_q['IND0230'] * -1 / ind_retro_q['IND0160'] *100) / 3
ind_retro_q['IND0300'] = (ind_retro_q['IND0240'] / ind_retro_q['IND0160'] *100) / 3
ind_retro_q['IND0310'] = (ind_retro_q['IND0250'] / ind_retro_q['IND0160'] *100) / 3
ind_retro_q['IND0320'] = ind_retro_q['IND0040'] - (ind_retro_q['IND0290'] + ind_retro_q['IND0300'] \
                                                   + ind_retro_q['IND0310'])
ind_retro_q['IND0330'] = ind_retro_q['IND0010'] + ind_retro_q['IND0290']+ind_retro_q['IND0300'] \
                        + ind_retro_q['IND0310'] + ind_retro_q['IND0320']
ind_retro_q['IND0340'] = ind_retro_q['IND0290'] / (100 - ind_retro_q['IND0030']) * 100
ind_retro_q['IND0350'] = ind_retro_q['IND0300'] / (100 - ind_retro_q['IND0030']) * 100
ind_retro_q['IND0360'] = ind_retro_q['IND0310'] / (100 - ind_retro_q['IND0030']) * 100
ind_retro_q['IND0370'] = ind_retro_q['IND0010'] / (100 - ind_retro_q['IND0030']) * 100
ind_retro_q['IND0380'] = ind_retro_q['IND0320'] / (100 - ind_retro_q['IND0030']) * 100
ind_retro_q['IND0390'] = ind_retro_q['IND0340'] + ind_retro_q['IND0350'] + ind_retro_q['IND0360'] \
                        + ind_retro_q['IND0370'] + ind_retro_q['IND0380'] 
ind_retro_q['IND0410'] = ind_retro_q['IND0060'] - ind_retro_q['IND0390'] 

In [378]:
check_rs = round(ind_retro_q['IND0410'], 2)
check_rs.loc['SUM'] = check_rs[:].sum()

In [383]:
if 0 in check_rs.loc['SUM']:
    print('spam')
    

In [315]:
ind_retro_q['IND0170'] = ind_retro_m['IND0170'].loc[:,ind_retro_m['IND0170'].columns.month.isin([1, 4, 7, 10])]
ind_retro_q['IND0170'].columns = ind_retro_q['IND0170'].columns - pd.DateOffset(months=1)

In [316]:
ind_retro_q['IND0170']

,2016-12-01,2017-03-01,2017-06-01,2017-09-01,2017-12-01,2018-03-01,2018-06-01,2018-09-01,2018-12-01,2019-03-01,...,2020-09-01,2020-12-01,2021-03-01,2021-06-01,2021-09-01,2021-12-01,2022-03-01,2022-06-01,2022-09-01,2022-12-01
0,10773733.0,10837490.0,11151067.0,11629176.0,12135449.0,12530011.0,13252498.0,14133803.0,14856625.0,15449315.0,...,19284924.0,19925665.0,20813283.0,22246726.0,23532034.0,24479744.0,25193667.0,24993365.0,25946336.0,26869679.0
1,3170420.0,3190932.0,3296435.0,3434582.0,3580420.0,3695252.0,3913072.0,4183151.0,4436861.0,4636061.0,...,5866106.0,6071985.0,6364966.0,6801054.0,7243508.0,7555402.0,7804143.0,7757172.0,8071071.0,8302535.0
2,90656.0,91480.0,94214.0,98197.0,102195.0,105184.0,110866.0,118048.0,123972.0,128826.0,...,161584.0,168022.0,175317.0,186252.0,197242.0,204257.0,208759.0,205305.0,212200.0,219813.0
3,62764.0,63287.0,65154.0,68012.0,71167.0,73323.0,77354.0,82616.0,86914.0,89866.0,...,110315.0,114207.0,118200.0,125527.0,132509.0,137402.0,140099.0,138097.0,142677.0,147353.0
4,77182.0,77982.0,80249.0,83634.0,87160.0,90275.0,95140.0,101152.0,106880.0,110683.0,...,135951.0,140858.0,146152.0,155556.0,164187.0,170512.0,174288.0,172145.0,177725.0,183714.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,67185.0,67179.0,67939.0,70408.0,72444.0,74341.0,78103.0,82577.0,86118.0,89162.0,...,112152.0,116794.0,122558.0,131131.0,139448.0,144839.0,150829.0,151523.0,158071.0,166354.0
92,18802.0,18876.0,19271.0,20289.0,20924.0,21372.0,22332.0,23859.0,24960.0,25909.0,...,31820.0,32953.0,34495.0,36574.0,39286.0,41160.0,41957.0,41322.0,43289.0,44697.0
93,49631.0,49921.0,51495.0,53792.0,56314.0,57795.0,61295.0,65975.0,69192.0,71646.0,...,94860.0,98610.0,103694.0,112777.0,121256.0,125559.0,128354.0,128173.0,132684.0,136644.0
94,10704.0,10714.0,10848.0,11231.0,11613.0,11952.0,12563.0,13402.0,14016.0,14537.0,...,17820.0,18262.0,18855.0,19833.0,20815.0,21338.0,21898.0,21752.0,22510.0,23359.0


In [304]:
ind_retro_q['IND0170']

,2017-01-01,2017-04-01,2017-07-01,2017-10-01,2018-01-01,2018-04-01,2018-07-01,2018-10-01,2019-01-01,2019-04-01,...,2020-10-01,2021-01-01,2021-04-01,2021-07-01,2021-10-01,2022-01-01,2022-04-01,2022-07-01,2022-10-01,2023-01-01
0,10773733.0,10837490.0,11151067.0,11629176.0,12135449.0,12530011.0,13252498.0,14133803.0,14856625.0,15449315.0,...,19284924.0,19925665.0,20813283.0,22246726.0,23532034.0,24479744.0,25193667.0,24993365.0,25946336.0,26869679.0
1,3170420.0,3190932.0,3296435.0,3434582.0,3580420.0,3695252.0,3913072.0,4183151.0,4436861.0,4636061.0,...,5866106.0,6071985.0,6364966.0,6801054.0,7243508.0,7555402.0,7804143.0,7757172.0,8071071.0,8302535.0
2,90656.0,91480.0,94214.0,98197.0,102195.0,105184.0,110866.0,118048.0,123972.0,128826.0,...,161584.0,168022.0,175317.0,186252.0,197242.0,204257.0,208759.0,205305.0,212200.0,219813.0
3,62764.0,63287.0,65154.0,68012.0,71167.0,73323.0,77354.0,82616.0,86914.0,89866.0,...,110315.0,114207.0,118200.0,125527.0,132509.0,137402.0,140099.0,138097.0,142677.0,147353.0
4,77182.0,77982.0,80249.0,83634.0,87160.0,90275.0,95140.0,101152.0,106880.0,110683.0,...,135951.0,140858.0,146152.0,155556.0,164187.0,170512.0,174288.0,172145.0,177725.0,183714.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,67185.0,67179.0,67939.0,70408.0,72444.0,74341.0,78103.0,82577.0,86118.0,89162.0,...,112152.0,116794.0,122558.0,131131.0,139448.0,144839.0,150829.0,151523.0,158071.0,166354.0
92,18802.0,18876.0,19271.0,20289.0,20924.0,21372.0,22332.0,23859.0,24960.0,25909.0,...,31820.0,32953.0,34495.0,36574.0,39286.0,41160.0,41957.0,41322.0,43289.0,44697.0
93,49631.0,49921.0,51495.0,53792.0,56314.0,57795.0,61295.0,65975.0,69192.0,71646.0,...,94860.0,98610.0,103694.0,112777.0,121256.0,125559.0,128354.0,128173.0,132684.0,136644.0
94,10704.0,10714.0,10848.0,11231.0,11613.0,11952.0,12563.0,13402.0,14016.0,14537.0,...,17820.0,18262.0,18855.0,19833.0,20815.0,21338.0,21898.0,21752.0,22510.0,23359.0


In [300]:
ind_retro_q['IND0260'] = ind_retro_q['IND0230'].copy()
ind_retro_q['IND0260'].loc[:,:] = np.nan
for ind in range (3, len)

In [301]:
ind_retro_q['IND0260']

,2017-03-01,2017-06-01,2017-09-01,2017-12-01,2018-03-01,2018-06-01,2018-09-01,2018-12-01,2019-03-01,2019-06-01,...,2020-12-01,2021-03-01,2021-06-01,2021-09-01,2021-12-01,2022-03-01,2022-06-01,2022-09-01,2022-12-01,2023-03-01
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
ind_retro_q['IND0140'].loc[:,ind_retro_q['IND0140'].columns.year == 2019] *\
ind_retro_y['IND0150'].loc[:,ind_retro_y['IND0150'].columns.year==2019].to_numpy()/ 1000000

ValueError: Unable to coerce to DataFrame, shape must be (96, 4): given (96, 0)

In [69]:
ind_retro_q['IND0140'].columns.year ==2018

array([ True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [57]:
ind_retro_q['IND0160']  = ind_retro_q['IND0140'] * ind_retro_y['IND0150'] / 1000000

KeyError: 'IND0150'

In [192]:
ind_retro_q['IND0020']

,IND,region_code,2018-03-01 00:00:00,2018-06-01 00:00:00,2018-09-01 00:00:00,2018-12-01 00:00:00,2019-03-01 00:00:00,2019-06-01 00:00:00,2019-09-01 00:00:00,2019-12-01 00:00:00,...,2020-09-01 00:00:00,2020-12-01 00:00:00,2021-03-01 00:00:00,2021-06-01 00:00:00,2021-09-01 00:00:00,2021-12-01 00:00:00,2022-03-01 00:00:00,2022-06-01 00:00:00,2022-09-01 00:00:00,2022-12-01 00:00:00
0,IND0020,0,83.5,80.5,84.9,75.70,86.3,80.4,83.7,75.1,...,80.4,70.3,87.7,80.8,81.8,74.5,92.7,76.2,77.9,68.8
1,IND0020,1,82.8,78.9,83.9,75.90,83.8,78.0,82.8,74.9,...,77.5,69.2,82.8,77.7,79.0,74.4,85.9,71.9,74.0,64.9
2,IND0020,2,81.7,87.4,82.3,71.20,91.7,81.4,81.8,71.5,...,80.5,75.2,92.4,82.1,79.5,74.0,93.5,77.2,78.0,67.0
3,IND0020,3,87.9,81.2,87.5,87.60,90.9,83.6,85.8,85.4,...,86.3,84.2,95.2,92.8,84.0,79.5,98.8,88.0,84.1,75.9
4,IND0020,4,87.5,83.2,87.2,79.00,89.8,81.4,84.6,76.0,...,81.3,73.7,94.4,86.8,83.3,76.9,97.6,79.9,81.5,73.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,IND0020,91,85.0,81.4,85.3,72.20,85.3,79.5,80.1,73.9,...,81.2,72.7,86.7,78.3,79.8,71.7,87.4,79.7,84.1,72.1
92,IND0020,92,62.3,56.9,59.1,57.80,61.4,53.6,57.4,54.2,...,46.4,46.1,82.7,74.9,53.6,48.8,86.9,76.7,55.2,45.0
93,IND0020,93,88.5,80.9,84.9,70.09,85.6,77.1,80.7,70.1,...,76.4,68.9,83.1,73.5,77.3,71.3,83.5,73.1,74.9,62.9
94,IND0020,94,77.2,76.3,82.5,73.59,87.6,77.3,80.3,71.5,...,78.7,69.5,89.8,84.3,79.0,67.8,93.3,81.4,78.5,67.5


In [119]:
# Удаляем из исходника все столбцы с датами которые есть в новом файле
# и дописываем новый файл в конец tuple
ind_retro_q['IND0020'].drop(columns=ind_goods_serv_q.columns, 
                            inplace=True)
ind_retro_q['IND0020'] = pd.concat([ind_retro_q['IND0020'], 
                                    ind_goods_serv_q], axis=1)

NameError: name 'ind_goods_serv_q' is not defined

## Банковский сектор

In [ ]:
deposits_rub = pd.read_excel("D:/ECONOMICS/savings/src1/Вклады.xlsx", 
                            header=1, 
                            sheet_name='в рублях')
deposits_cur = pd.read_excel("D:/ECONOMICS/savings/src1/Вклады.xlsx", 
                            header=2, 
                            sheet_name='в инвалюте')
deposits_sum = pd.read_excel("D:/ECONOMICS/savings/src1/Вклады.xlsx", 
                            header=1, 
                            sheet_name='итого')

In [ ]:
def format_fin_sectror(df):
    df.drop(columns=df.columns[0], inplace=True)
    df = df.iloc[:96,:]
    df.dropna(axis=1, how='all',inplace=True)
    df.columns = pd.to_datetime(df.columns, 
                                format="%d.%m.%Y", 
                                errors='coerce')
    return df

In [ ]:
deposits_rub = format_fin_sectror(deposits_rub)
deposits_cur = format_fin_sectror(deposits_cur)
deposits_sum = format_fin_sectror(deposits_sum)

In [ ]:
deposits_rub

In [ ]:
deposits_rub

In [ ]:
deposits_rub.drop(columns='Unnamed: 0', inplace=True)
deposits_rub = deposits_rub.iloc[:96,:]
#deposits_rub.columns = deposits_rub.columns.astype('datetime64[ns]')
deposits_rub

In [ ]:
#list(deposits_rub.columns)
deposits_rub.dropna(axis=1, how='all',inplace=True)
deposits_rub.columns = pd.to_datetime(deposits_rub.columns, format="%d.%m.%Y", errors='coerce')
deposits_rub.iloc[:,5:]

In [ ]:
deposits_rub.iloc[:,5:]

In [ ]:
columns_date =pd.Series(deposits_rub.columns.str.split('.')).\
                apply(lambda x: '-'.join(x[::-1]))
#deposits_rub.columns = deposits_rub.columns.str.replace('.','-')
# aa = pd.DataFrame(deposits_rub.columns)
# aa = aa[0].apply(lambda x: x[::-1])
#deposits_rub.columns = deposits_rub.columns.astype('datetime64[M]')

In [ ]:
deposits_rub.columns = pd.to_datetime(deposits_rub.columns, format="%d.%m.%Y", errors='coerce')

In [ ]:
deposits_rub.columns.isna()

In [ ]:
deposits_rub.columns = columns_date

In [ ]:
deposits_rub.columns = deposits_rub.columns.astype('str')
deposits_rub.columns = deposits_rub.columns.astype('datetime64[ns]')

In [ ]:
deposits_rub

In [ ]:
columns_date.apply(lambda x: '-'.join(x[::-1]))

In [ ]:
deposits_rub.columns.str.split('-')

In [ ]:
dates = pd.Series(deposits_rub.columns)
dates.replace(to_replace='.', value='-', inplace=True)
dates
#deposits_rub.columns = deposits_rub.columns.astype('datetime64[ns]')

In [ ]:
ind_cash_q['2020'].head(5)

In [ ]:
#ind_cash_y.loc[1,['region_id', 'state_id']] = [99.0,99.0]

In [ ]:
ind_cash_y[ind_cash_y.region.str.contains('Российска')]

In [ ]:
regions_dict

In [ ]:
ind_cash_q.iloc[:,0]

In [ ]:
ind_cash_q.droplevel('Unnamed: 1_level_1', axis=1)
kk


In [1]:
import scipy

In [ ]:
ar = Ri